In [21]:
# load dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [22]:
# Load data and select trainging hour and test hour
trainhour = 5
testhour = 7
df = pd.read_csv('series.csv')
cols5 = ['21', '22', '23', '24', '25']
cols10 = ['21', '22', '23', '24', '25', '26', '27', '28', '29', '30']
df['sum5'] = df[cols5].sum(axis=1)
df['sum10'] = df[cols10].sum(axis=1)
data = df.drop(df[(df['hour'] != trainhour)].index)
testdata = df.drop(df[(df['hour'] != testhour)].index)
del df

In [23]:
# Categorize the 'sum' column the number of PIPs over selected range
def setlabels(x):
    if x < -10:
        return -1
    elif x > 10:
        return 1
    else:
        return 0

data["labels"] = data["sum10"].apply(setlabels)
testdata["labels"] = testdata["sum10"].apply(setlabels)

In [24]:
# Get X, y for model
X = data.iloc[:,8:28].to_numpy()

from sklearn.preprocessing import LabelEncoder
y = data['labels']
label_encoder = LabelEncoder()
label_encoder.fit(y)
y_encoded = label_encoder.transform(y)
y = y_encoded

In [25]:
# Split data into training and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [26]:
# Create model and train model
from sklearn.svm import SVC 
model = SVC(kernel='poly')
model.fit(X_train, y_train)
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

C:\Users\mryet\Anaconda3\envs\PythonData\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.9801750772399588
0.8892893923789907


In [27]:
# Assign resutls to fitted data
data['class'] = label_encoder.inverse_transform(model.predict(X))
data_group = data.groupby('labels').agg(['sum','count', 'min', 'max'])
data_group['sum10']

,sum,count,min,max
labels,,,,
-1,-4995.3,364,-38.7,-10.1
0,-1102.6,14838,-10.0,10.0
1,4937.9,334,10.0,45.9


In [28]:
# Pull out validation data and make predicts
formodel = testdata.iloc[:,8:28].to_numpy()
testdata['class'] = label_encoder.inverse_transform(model.predict(formodel))
test_group = testdata.groupby('class').agg(['sum','count', 'min', 'max'])
test_group['sum10']
testdata = testdata.drop(testdata[(testdata['class'] == 0)].index)

In [29]:
#testdata = testdata.drop('profit',axis=1, inplace=True)
totalrows = testdata.shape[0]
totalcols = testdata.shape[1]
profit = []
for i in range(totalrows):
    temp = 0
    for j in range(10):
        temp += testdata.iloc[i,28+j]
        if (testdata.iloc[i,totalcols-1] == -1):
            if(temp > 2):
              break  
        else:
            if(temp < -2):
              break
    
    if (testdata.iloc[i,totalcols-1] == -1):
        temp = - temp

    profit.append(temp)

totalrows = testdata.shape[0]
totalcols = testdata.shape[1]
profit = []
for i in range(totalrows):
    temp = 0
    for j in range(10):
        temp += testdata.iloc[i,28+j]
        if (testdata.iloc[i,totalcols-1] == -1):
            if(temp > 2):
              break  
        else:
            if(temp < -2):
              break
    
    if (testdata.iloc[i,totalcols-1] == -1):
        temp = -temp
    profit.append(temp)

print("Est Profit: " + str(sum(profit)))

Est Profit: -441.59999999999997


In [30]:
testdata.to_csv('binarytest.csv')